# Sektöre Göre Göreli Hisse Performans Analizi

Bu çalışmada şirket fiyatları yalnız başına değil, sektör ortalamalarına göre değerlendirilmiştir.  
Amaç ham fiyat yerine göreli performansı ve trend davranışını incelemektir.

(Örnek veri seti kullanılarak sektör bazlı göreli performans analizi gerçekleştirilmiştir.)

In [2]:
data = {
    "Date": [
        "01.01.2024","02.01.2024","03.01.2024","04.01.2024",
        "05.01.2024","06.01.2024","07.01.2024","08.01.2024",
        "09.01.2024","10.01.2024","11.01.2024","12.01.2024",
        "13.01.2024","14.01.2024","15.01.2024"
    ],
    "Company":[
        "A","A","A","B","B","B","C","C","C","A","B","C","A","B","C"
    ],
    "Sector":[
        "Tech","Tech","Tech",
        "Finance","Finance","Finance",
        "Energy","Energy","Energy",
        "Tech","Finance","Energy",
        "Tech","Finance","Energy"
    ],
    "Price":[100,110,105,80,82,90,70,75,72,120,88,78,130,92,80],
    "Volume":[200,220,210,150,160,155,300,320,310,240,165,305,260,170,315],
    "Change %":[
        "1.2%","2.1%","-0.5%","0.8%","1.1%","2.0%",
        "-1.2%","0.5%","1.0%","3.1%","0.9%","2.2%",
        "1.7%","1.3%","0.4%"
    ]
}

In [3]:
import pandas as pd
df = pd.DataFrame(data)
df.head()

,Date,Company,Sector,Price,Volume,Change %
0,01.01.2024,A,Tech,100,200,1.2%
1,02.01.2024,A,Tech,110,220,2.1%
2,03.01.2024,A,Tech,105,210,-0.5%
3,04.01.2024,B,Finance,80,150,0.8%
4,05.01.2024,B,Finance,82,160,1.1%


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Date      15 non-null     object
 1   Company   15 non-null     object
 2   Sector    15 non-null     object
 3   Price     15 non-null     int64 
 4   Volume    15 non-null     int64 
 5   Change %  15 non-null     object
dtypes: int64(2), object(4)
memory usage: 852.0+ bytes


## Veri Hazırlama

Analiz öncesinde veri tipleri düzenlenmiş ve zaman bazlı analiz yapılabilmesi için gerekli dönüşümler uygulanmıştır.

In [ ]:
df["Date"] = pd.to_datetime(df["Date"],dayfirst=True)
df["Change %"] = df["Change %"].str.rstrip("%").astype(float)
df = df.sort_values("Date")
df["Year"] = df["Date"].dt.year
df["Month"] = df["Date"].dt.month
df.head()

,Date,Company,Sector,Price,Volume,Change %,Year,Month
0,2024-01-01,A,Tech,100,200,1.2,2024,1
1,2024-01-02,A,Tech,110,220,2.1,2024,1
2,2024-01-03,A,Tech,105,210,-0.5,2024,1
3,2024-01-04,B,Finance,80,150,0.8,2024,1
4,2024-01-05,B,Finance,82,160,1.1,2024,1


## Sektör Bazlı Analiz

Şirket performansı sektör ortalamalarına göre karşılaştırılarak göreli fiyat ve işlem payı metrikleri oluşturulmuştur.

In [6]:
sector_kpi = df.groupby("Sector").agg(Price_Mean=("Price","mean"), Price_Max = ("Price","max"), Volume_Mean=("Volume","mean"), Change_Mean =("Change %","mean"))
print(sector_kpi)

         Price_Mean  Price_Max  Volume_Mean  Change_Mean
Sector                                                  
Energy         75.0         80        310.0         0.58
Finance        86.4         92        160.0         1.22
Tech          113.0        130        226.0         1.52


In [7]:
df["Sector_Mean_Price"] = df.groupby("Sector")["Price"].transform("mean")
df["Relative_Price"] = df["Price"] / df["Sector_Mean_Price"]
df["Volume_Share"] = df["Volume"] / df.groupby("Sector")["Volume"].transform("sum")
df.head(8)

,Date,Company,Sector,Price,Volume,Change %,Year,Month,Sector_Mean_Price,Relative_Price,Volume_Share
0,2024-01-01,A,Tech,100,200,1.2,2024,1,113.0,0.884956,0.176991
1,2024-01-02,A,Tech,110,220,2.1,2024,1,113.0,0.973451,0.194690
2,2024-01-03,A,Tech,105,210,-0.5,2024,1,113.0,0.929204,0.185841
3,2024-01-04,B,Finance,80,150,0.8,2024,1,86.4,0.925926,0.187500
4,2024-01-05,B,Finance,82,160,1.1,2024,1,86.4,0.949074,0.200000
5,2024-01-06,B,Finance,90,155,2.0,2024,1,86.4,1.041667,0.193750
6,2024-01-07,C,Energy,70,300,-1.2,2024,1,75.0,0.933333,0.193548
7,2024-01-08,C,Energy,75,320,0.5,2024,1,75.0,1.000000,0.206452


## Trend Analizi

Kısa ve uzun dönem hareketli ortalamalar kullanılarak fiyat momentumunun yönü belirlenmiştir.

In [8]:
df["MA_3"] = df["Price"].rolling(3).mean()
df["MA_5"] = df["Price"].rolling(5).mean()
df["Trend_Up"] = df["MA_3"] > df["MA_5"]
df.head(3)

,Date,Company,Sector,Price,Volume,Change %,Year,Month,Sector_Mean_Price,Relative_Price,Volume_Share,MA_3,MA_5,Trend_Up
0,2024-01-01,A,Tech,100,200,1.2,2024,1,113.0,0.884956,0.176991,NaN,NaN,False
1,2024-01-02,A,Tech,110,220,2.1,2024,1,113.0,0.973451,0.194690,NaN,NaN,False
2,2024-01-03,A,Tech,105,210,-0.5,2024,1,113.0,0.929204,0.185841,105.0,NaN,False


In [9]:
print(f"Toplam Trend Gün sayısı: {df["Trend_Up"].sum()}")

Toplam Trend Gün sayısı: 5


In [10]:
trend_df = df[df["Trend_Up"]== True]
print(trend_df)

         Date Company   Sector  Price  Volume  Change %  Year  Month  \
9  2024-01-10       A     Tech    120     240       3.1  2024      1   
10 2024-01-11       B  Finance     88     165       0.9  2024      1   
11 2024-01-12       C   Energy     78     305       2.2  2024      1   
12 2024-01-13       A     Tech    130     260       1.7  2024      1   
14 2024-01-15       C   Energy     80     315       0.4  2024      1   

    Sector_Mean_Price  Relative_Price  Volume_Share        MA_3  MA_5  \
9               113.0        1.061947      0.212389   89.000000  85.4   
10               86.4        1.018519      0.206250   93.333333  85.0   
11               75.0        1.040000      0.196774   95.333333  86.6   
12              113.0        1.150442      0.230088   98.666667  97.6   
14               75.0        1.066667      0.203226  100.666667  93.6   

    Trend_Up  
9       True  
10      True  
11      True  
12      True  
14      True  


In [11]:
trend_df.groupby("Sector").agg(Trend_Price_Mean = ("Price","mean"),Trend_Volume_Mean= ("Volume","mean"),Count= ("Sector","size")).sort_values("Sector",ascending= False)

,Trend_Price_Mean,Trend_Volume_Mean,Count
Sector,,,
Tech,125.0,250.0,2
Finance,88.0,165.0,1
Energy,79.0,310.0,2


In [12]:
df.sort_values("Relative_Price",ascending= False)[["Date","Company","Sector","Price","Sector_Mean_Price","Relative_Price"]].head(3)   

,Date,Company,Sector,Price,Sector_Mean_Price,Relative_Price
12,2024-01-13,A,Tech,130,113.0,1.150442
14,2024-01-15,C,Energy,80,75.0,1.066667
13,2024-01-14,B,Finance,92,86.4,1.064815


## Sonuç

Göreli fiyat yaklaşımı şirketlerin yalnız başına değil sektör bağlamında değerlendirilmesini sağlamıştır.

Analiz sonucunda trend oluşan günlerde fiyatların sektör ortalamasının üzerine çıktığı gözlemlenmiştir. Bu yaklaşım momentum tespiti ve karşılaştırmalı piyasa analizi için kullanılabilir.